### Second Map Crime Development with Dynamic Time Selection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import calplot
import folium
from folium.plugins import HeatMap
import json
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Legend, Select
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
from bokeh.io import output_notebook, show
from bokeh.palettes import Category10
from bokeh.layouts import gridplot, column

In [ ]:
# Load the data
crime_data = pd.read_csv("police_data.csv")
1
# Load the data
2
crime_data = pd.read_csv("police_data.csv")
1
# Load the map data
2
with open("sfpd.geojson") as f:
3
    geo_json_data = json.load(f)
1
# Create the Period column
2
crime_data['Period'] = pd.to_datetime(crime_data['Date']).dt.strftime('%m/%d/%Y')
3
crime_data['Period'] = pd.to_datetime(crime_data['Period'], format='%m/%d/%Y')
4
bins = [pd.Timestamp('2002-01-01'), pd.Timestamp('2007-12-31'), pd.Timestamp('2012-12-31'), pd.Timestamp('2018-05-13')]
5
labels = ['Early', 'Middle', 'Late']
6
crime_data['period'] = pd.cut(crime_data['Period'], bins=bins, labels=labels)
1
# Select the list of features
2
features = ['X', 'Y', 'DayOfWeek', 'Time', 'PdDistrict']
1
# Transfer features to numbers
2
crime_data = pd.get_dummies(crime_data, columns=['DayOfWeek', 'PdDistrict'])
1
# Create the model and train it
2
features = ['X', 'Y', 'DayOfWeek_Friday', 'DayOfWeek_Monday', 'DayOfWeek_Saturday',            'DayOfWeek_Sunday', 'DayOfWeek_Thursday', 'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday',            'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE', 'PdDistrict_MISSION',            'PdDistrict_NORTHERN', 'PdDistrict_PARK', 'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN',            'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN']
3
kmeans = KMeans(n_clusters=3)
4
kmeans.fit(crime_data[features])
KMeans(n_clusters=3)


In [3]:
# Load the map data
with open("sfpd.geojson") as f:
    geo_json_data = json.load(f)

In [36]:
# Create the Period column
crime_data['Period'] = pd.to_datetime(crime_data['Date']).dt.strftime('%m/%d/%Y')
crime_data['Period'] = pd.to_datetime(crime_data['Period'], format='%m/%d/%Y')
bins = [pd.Timestamp('2002-01-01'), pd.Timestamp('2007-12-31'), pd.Timestamp('2012-12-31'), pd.Timestamp('2018-05-13')]
labels = ['Early', 'Middle', 'Late']
crime_data['period'] = pd.cut(crime_data['Period'], bins=bins, labels=labels)

In [38]:
# Select the list of features
features = ['X', 'Y', 'DayOfWeek', 'Time', 'PdDistrict']

In [40]:
# Transfer features to numbers
crime_data = pd.get_dummies(crime_data, columns=['DayOfWeek', 'PdDistrict'])

In [43]:
# Create the model and train it
features = ['X', 'Y', 'DayOfWeek_Friday', 'DayOfWeek_Monday', 'DayOfWeek_Saturday',            'DayOfWeek_Sunday', 'DayOfWeek_Thursday', 'DayOfWeek_Tuesday', 'DayOfWeek_Wednesday',            'PdDistrict_BAYVIEW', 'PdDistrict_CENTRAL', 'PdDistrict_INGLESIDE', 'PdDistrict_MISSION',            'PdDistrict_NORTHERN', 'PdDistrict_PARK', 'PdDistrict_RICHMOND', 'PdDistrict_SOUTHERN',            'PdDistrict_TARAVAL', 'PdDistrict_TENDERLOIN']
kmeans = KMeans(n_clusters=3)
kmeans.fit(crime_data[features])

KMeans(n_clusters=3)

In [1]:
# Add the cluster labels

crime_data['cluster'] = kmeans.predict(crime_data[features])

# Label clusters as early, mid or late gentrification groups
early_cluster = crime_data[crime_data['cluster'] == 0]
mid_cluster = crime_data[crime_data['cluster'] == 1]
late_cluster = crime_data[crime_data['cluster'] == 2]

early_crime = early_cluster[early_cluster['period'] == 'Early']
mid_crime = mid_cluster[mid_cluster['period'] == 'Middle']
late_crime = late_cluster[late_cluster['period'] == 'Late']

# Create map
comparison_map = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# Add Heatmap
HeatMap(data=early_crime[['Y', 'X']], name='Early Period Crime Heatmap', min_opacity=0.2, max_zoom=18, radius=10, blur=15).add_to(comparison_map)
HeatMap(data=mid_crime[['Y', 'X']], name='Mid Period Crime Heatmap', min_opacity=0.2, max_zoom=18, radius=10, blur=15).add_to(comparison_map)
HeatMap(data=late_crime[['Y', 'X']], name='Late Period Crime Heatmap', min_opacity=0.2, max_zoom=18, radius=10, blur=15).add_to(comparison_map)

folium.LayerControl().add_to(comparison_map)


NameError: name 'kmeans' is not defined

In [ ]:
crime_data = pd.read_csv("police_data.csv")
# Extract date and time information from Date and Time columns
crime_data['Date'] = pd.to_datetime(crime_data['Date'])
crime_data['Time'] = pd.to_datetime(crime_data['Time'], format='%H:%M').dt.hour

# Filter data for 2010-2017
crime_data = crime_data[(crime_data['Date'] >= '2010-01-01') & (crime_data['Date'] < '2018-01-01')]

# Group by hour of day and crime category
crime_by_hour_category_district = crime_data.groupby(['Time', 'Category', 'PdDistrict']).size().reset_index(name='count')

# Normalize crime count by category and hour
crime_by_hour_category_district['normalized_count'] = crime_by_hour_category_district.groupby(['Category', 'PdDistrict'])['count'].apply(lambda x: x/x.sum())

# Set the list of hours for x-axis
hours = [str(i) for i in range(24)]

# Get the list of districts for subplots
districts = crime_by_hour_category_district['PdDistrict'].unique()

# Create a dictionary of sources for each district
sources = {}
for district in districts:
    sources[district] = ColumnDataSource(crime_by_hour_category_district[crime_by_hour_category_district['PdDistrict'] == district])

# Create Bokeh subplots for each district
plots = []
for district in districts:
    p = figure(title=district, x_range=FactorRange(factors=hours), plot_width=400, plot_height=300)

    # Create a color mapper for each crime category
    category_mapper = factor_cmap(field_name='Category', palette=Category10_10, factors=crime_by_hour_category_district['Category'].unique())

    # Add vbar for each crime category and hour
    bar = p.vbar(x='Time', top='normalized_count', source=sources[district], width=0.9, color=category_mapper, legend_label='Category', muted_alpha=0.2, muted_color=category_mapper)

    # Add legend with interactive click policy
    p.legend.click_policy="mute"

    # Set other figure properties
    p.title.text_font_size = '16pt'
    p.xaxis.axis_label = 'Hour of Day'
    p.yaxis.axis_label = 'Normalized Crime Count'

    plots.append(p)

# Create a gridplot of subplots
grid = gridplot([[plots[0], plots[1]], [plots[2], plots[3]]], plot_width=500, plot_height=400)

# Save the plot as an HTML file
output_file("crime_count_by_district.html")

# Display the plot
show(grid)


In [ ]:
crime_data = pd.read_csv("police_data.csv")

In [14]:
# read in the crime data

# convert Date and Time columns to datetime format
crime_data['Date'] = pd.to_datetime(crime_data['Date'], format='%m/%d/%Y')
crime_data['Time'] = pd.to_datetime(crime_data['Time'], format='%H:%M')

# group the data by district and year
district_year_data = crime_data.groupby(['PdDistrict', 'Category', pd.Grouper(key='Date', freq='Y')]).size().reset_index(name='count')

# create a dictionary of data sources for each district
district_sources = {}
for district in district_year_data['PdDistrict'].unique():
    district_data = district_year_data[district_year_data['PdDistrict'] == district]
    district_sources[district] = ColumnDataSource(district_data)

# create a dictionary of plots for each district
district_plots = {}
for district in district_year_data['PdDistrict'].unique():
    district_data = district_year_data[district_year_data['PdDistrict'] == district]
    source = district_sources[district]
    plot = figure(title=district, x_axis_type='datetime', height=250, width=400, y_range=(0, district_data['count'].max()))
    plot.vbar(x='Date', top='count', source=source, width=365*60*60*1000)
    district_plots[district] = plot

# create a Select widget with options for district and category
category_options = list(district_year_data['Category'].unique())
category_select = Select(title="Category:", options=category_options, value=category_options[0])

# define a callback function to update the plot based on the selected options
def update_plot(attr, old, new):
    category = category_select.value
    new_source = district_sources[district][district_sources[district]['Category'] == category]
    district_plots[district].title.text = f"({category})"
    district_plots[district].y_range.end = new_source.data['count'].max()
    district_plots[district].title.text = f"({category})"
    district_plots[district].title.align = 'center'
    district_plots[district].vbar(x='Date', top='count', source=new_source, width=365*60*60*1000)

# add the callback function to the Select widgets
category_select.on_change('value', update_plot)

# create a layout with the gridplot and Select widgets
layout = column(category_select, gridplot(list(district_plots.values()), ncols=2))

# Save the plot as an HTML file
output_file("crime_count_by_district.html")

# show the layout
show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



<font color=black size=4>**References**</font>

1."U.S. recession fuels crime rise, police chiefs say" https://www.reuters.com/article/us-usa-economy-crime-idUSTRE50Q6FR20090127

2."SF sees highest crime rate since 2008": https://www.sfexaminer.com/news/sf-sees-highest-crime-rate-since-2008/article_c705b09e-1eaa-5783-b367-17537458cdd6.html

3."Central Market Economic Strategy": https://commissions.sfplanning.org/cpcpackets/CentralMarketEconomicStrategyCPC.pdf